In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm

In [2]:
sys.path.append('../../datasets/human_activity_segmentation_challenge-main')
from utils import *
sys.path.append('../../benchmark/utils')
from evaluation import *
sys.path.append('../')
from ClassificationParScoreProfile import *
from DynamicProgramming import *
from FLOSS import *
from Pelt import *
from Window import *
from BayesianOnlineChangePointDetection import *
from Fluss import *

<h3>Load Dataset</h3>

In [3]:
# sys.path.append("../../datasets/human_activity_segmentation_challenge-main/datasets")
df = load_master_data("../../datasets/human_activity_segmentation_challenge-main/datasets/has2023_master.csv.zip")
df.head() 	

,ts_id,ts_challenge_id,group,subject,routine,change_points,activities,split,x-acc,y-acc,z-acc,x-gyro,y-gyro,z-gyro,x-mag,y-mag,z-mag,lat,lon,speed
0,0,125,indoor,1,1,"[1461, 2952, 4473, 5022, 6460]","[jumping jacks, downtoplank, jumping jacks, re...",private,"[0.406, 0.7786, 0.70785, 0.5649, 0.268, -0.035...","[-0.4804, -0.19365, 0.4704, 0.6985, 0.45065, 0...","[1.775, 2.5445, 2.3076499999999998, 1.85993333...","[-0.2425, -0.1857, -0.0446, 0.0733, -0.0122, -...","[1.2022, 1.204, 1.2584, 1.4465, 1.6316, 1.5009...","[-0.0538, -0.0324, 0.0147, 0.0568, 0.0531, -0....","[-24.9, -23.4, -23.4, -23.4, -21.3, -21.3, -21...","[60.3, 62.0, 62.0, 62.0, 61.2, 61.2, 61.2, 62....","[-24.4, -26.1, -26.1, -26.1, -26.9, -26.9, -26...",[],[],[]
1,1,140,indoor,1,1,"[772, 2056, 3521, 4835, 5283, 6893]","[rest, plank, sit ups, plank, rest, jumping ja...",public,"[-0.2274, -0.2274, 0.1261, 0.1261, 0.1261, -0....","[0.5969, 0.5969, 0.1466, 0.1466, 0.1466, 0.515...","[-0.5992, -0.5992, -0.0643, -0.0643, -0.0643, ...","[-0.15955714285714287, -0.1613, -0.10250000000...","[-0.045385714285714286, -0.0391, -0.0080599999...","[-0.1735142857142857, -0.1772, -0.195, -0.2097...","[-29.4, -28.7, -28.7, -28.7, -29.1, -29.1, -29...","[-63.5, -62.4, -62.4, -62.4, -62.7, -62.7, -62...","[-52.2, -52.4, -52.4, -52.4, -52.2, -52.2, -52...",[],[],[]
2,2,93,indoor,1,1,"[1211, 2043, 2836, 3529, 4036]","[plank, squat jump, lunges left, lunges right,...",public,"[4.8883, 4.8883, 4.8883, 4.4134, 4.4134, 4.413...","[5.4784, 5.4784, 5.4784, 5.8291, 5.8291, 5.829...","[-6.4445, -6.4445, -6.4445, -6.0946, -6.0946, ...","[0.060823076923076924, 0.08336, 0.0929, 0.2059...","[0.01392307692307692, -0.00486, -0.0012, 0.018...","[0.054430769230769234, 0.04057, 0.0391, 0.0824...","[-34.2, -34.2, -34.0, -34.0, -34.0, -34.0, -34...","[3.0, 3.0, 2.8, 2.8, 2.8, 4.3, 4.3, 4.3, 4.0, ...","[63.9, 63.9, 64.8, 64.8, 64.8, 64.9, 64.9, 64....",[],[],[]
3,3,52,indoor,1,1,"[739, 1433, 2044, 2875, 3589]","[lunges left, lunges right, rest, squat jump, ...",private,"[-1.2656, -1.2656, -1.2656, -6.6303, -6.6303, ...","[-0.5253, -0.5253, -0.5253, -0.0624, -0.0624, ...","[-3.9209, -3.9209, -3.9209, -4.8733, -4.8733, ...","[-0.47177142857142856, -0.46897142857142854, -...","[0.7421142857142857, 0.44257142857142856, 0.26...","[-0.16341428571428573, -0.4535285714285714, -0...","[-1.7, -2.9, -2.9, -2.9, -4.9, -4.9, -4.9, -5....","[76.8, 75.4, 75.4, 75.4, 74.8, 74.8, 74.8, 74....","[-5.0, -3.7, -3.7, -3.7, -1.2, -1.2, -1.2, 1.5...",[],[],[]
4,4,221,indoor,1,1,"[1506, 3033, 4477, 5348, 6805]","[hiptrust, diagonal, hiptrust, rest, diagonal,...",private,"[-0.8136, -0.8136, -0.8136, -0.1624, -0.1624, ...","[-0.1392, -0.1392, -0.1392, -1.0283, -1.0283, ...","[0.844, 0.844, 0.844, 1.1861, 1.1861, 1.1861, ...","[-0.654675, -0.7183900000000001, -0.7294, -0.7...","[-0.021399999999999995, -0.11534, -0.1124, 0.0...","[0.032975, 0.05033, 0.0452, 0.03444, 0.0851099...","[10.6, 10.6, 10.6, 11.3, 11.3, 11.3, 12.1, 12....","[-69.2, -69.2, -69.2, -64.5, -64.5, -64.5, -60...","[-50.3, -50.3, -50.3, -53.5, -53.5, -53.5, -57...",[],[],[]


In [4]:
cols = [5,14]
df=df[df.columns[cols]]
df.head()

,change_points,x-mag
0,"[1461, 2952, 4473, 5022, 6460]","[-24.9, -23.4, -23.4, -23.4, -21.3, -21.3, -21..."
1,"[772, 2056, 3521, 4835, 5283, 6893]","[-29.4, -28.7, -28.7, -28.7, -29.1, -29.1, -29..."
2,"[1211, 2043, 2836, 3529, 4036]","[-34.2, -34.2, -34.0, -34.0, -34.0, -34.0, -34..."
3,"[739, 1433, 2044, 2875, 3589]","[-1.7, -2.9, -2.9, -2.9, -4.9, -4.9, -4.9, -5...."
4,"[1506, 3033, 4477, 5348, 6805]","[10.6, 10.6, 10.6, 11.3, 11.3, 11.3, 12.1, 12...."


In [5]:
results={}
def evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,algorithm_name):
    # Initialize lists to store metric values for each time series
    covering_scores = []
    f_measure_scores = []
    nab_scores = []
    i=0
    # Iterate over each time series
    for cps_true,cps_found,ts_len in zip(all_cps_true,all_cps_found,all_ts_len):
        i=i+1
        score_covering = covering({0: cps_true}, cps_found, ts_len)
        covering_scores.append(score_covering)
        
        # Calculate F-measure
        score_F = f_measure({0: cps_true}, cps_found, ts_len) 
        f_measure_scores.append(score_F)
        
        # Format true and predicted change points as Series
        true_cp, predicted_cp = formate_data(cps_true, cps_found,ts_len)
    
        # Evaluate change points using the evaluating_change_point function
        score_NAB = evaluating_change_point(true_cp, predicted_cp, metric='nab')
        nab_scores.append(score_NAB)
        # print(score_NAB)

    # Calculate mean scores across all time series
    mean_covering = sum(covering_scores) / len(covering_scores)
    mean_f_measure = sum(f_measure_scores) / len(f_measure_scores)

    # Calculate mean NAB scores
    nab_scores_filtered = [score for score in nab_scores if not np.isnan(score['Standart']) and not np.isinf(score['Standart'])]
    mean_nab_standard = sum(score['Standart'] for score in nab_scores_filtered) / len(nab_scores_filtered)
    mean_nab_lowFP = sum(score['LowFP'] for score in nab_scores_filtered) / len(nab_scores_filtered)
    mean_nab_lowFN = sum(score['LowFN'] for score in nab_scores_filtered) / len(nab_scores_filtered)


    # Print mean scores
    print(f"Mean Covering: {mean_covering}")
    print(f"Mean F-measure: {mean_f_measure}")
    print(f"Mean NAB (Standard): {mean_nab_standard}")
    print(f"Mean NAB (LowFP): {mean_nab_lowFP}")
    print(f"Mean NAB (LowFN): {mean_nab_lowFN}")

    results[algorithm_name]={"Covering":mean_covering,
                                        "F-measure":mean_f_measure,
                                        "NAB (Standard)":mean_nab_standard,
                                        "NAB (LowFP)":mean_nab_lowFP,
                                        "NAB (LowFN)":mean_nab_lowFN
    }


<h3>BinaryClaSPSegmentation</h3>

In [6]:
all_cps_true=[]
all_cps_found=[]
all_ts_len=[]
for _,(cps_true,ts) in tqdm(df.iterrows()):
  cps_found = BinaryClaSPSegmentation().fit_predict(ts)
  all_cps_true.append(cps_true)
  all_cps_found.append(cps_found)
  all_ts_len.append(ts.shape[0])

evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,"BinaryClaSPSegmentation")


118it [11:38,  6.76s/it]d:\Time_Series_Segmenation\benchmark\Evaluating\..\ClassificationParScoreProfile.py:87: UserWarning: Time series must at least have 2*min_seg_size data points for segmentation. Try setting a smaller window size.
  warnings.warn(
152it [25:12, 68.55s/it]

<h3>DynamicProgramming</h3>

In [ ]:
all_cps_true=[]
all_cps_found=[]
all_ts_len=[]
for _,(cps_true,ts) in tqdm(df.iterrows()):
  cps_found = DynamicProgramming().fit_predict(np.array(ts),len(cps_true))[:-1]
  all_cps_true.append(cps_true)
  all_cps_found.append(cps_found)
  all_ts_len.append(ts.shape[0])

evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,"DynamicProgramming")

0it [05:18, ?it/s]


KeyboardInterrupt: 

<h3>FLOSS</h3>

In [ ]:
all_cps_true=[]
all_cps_found=[]
all_ts_len=[]
for _,(cps_true,ts) in tqdm(df.iterrows()):
    floss_detector = FLOSS(n_timepoints=len(ts), window_size=20, n_prerun=500, threshold=0.7, excl_factor=5, verbose=0)
    # Update FLOSS with each time point
    for timepoint in ts:
        floss_detector.update(timepoint)

    cps_found = floss_detector.change_points
    all_cps_true.append(cps_true)
    all_cps_found.append(cps_found)
    all_ts_len.append(ts.shape[0])

evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,"FLOSS")


Time Series: 0: True Change Points: [1461 2952 4473 5022 6460], Found Change Points: [198, 726, 966, 1088, 1207, 1405, 1641, 2000, 2963, 3084, 3443, 3890, 3756, 4128, 4416, 4547, 4969, 5299, 5650, 6188, 6413, 6515, 6700], Score: 0.49839917375386705
Time Series: 1: True Change Points: [ 772 2056 3521 4835 5283 6893], Found Change Points: [596, 763, 1374, 1525, 2035, 2795, 3421, 3617, 4050, 4525, 5034, 5146, 5276, 5463, 5992, 6255, 6518, 6838, 6949, 7051], Score: 0.5204521335378668
Time Series: 2: True Change Points: [1211 2043 2836 3529 4036], Found Change Points: [1186, 1577, 1982, 2092, 2931, 3060, 3275, 3587, 3982, 4134], Score: 0.748345869074771
Time Series: 3: True Change Points: [ 739 1433 2044 2875 3589], Found Change Points: [827, 964, 1438, 1571, 2006, 2815, 2925, 3078, 3239, 3482], Score: 0.7350647878676698
Time Series: 4: True Change Points: [1506 3033 4477 5348 6805], Found Change Points: [962, 1225, 1366, 1699, 2420, 2565, 2843, 3303, 3583, 3699, 4228, 4345, 4505, 4816, 517

<h3>Pelt</h3>

In [ ]:
all_cps_true=[]
all_cps_found=[]
all_ts_len=[]
for _,(cps_true,ts) in tqdm(df.iterrows()):
  cps_found = Pelt().fit_predict(np.array(ts),pen=45)
  all_cps_true.append(cps_true)
  all_cps_found.append(cps_found)
  all_ts_len.append(ts.shape[0])

evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,"Pelt")

Time Series: 0: True Change Points: [1461 2952 4473 5022 6460], Found Change Points: [1490, 1550, 1640, 1705, 1745, 1840, 1930, 1990, 2050, 2125, 2230, 2300, 2355, 2430, 2525, 2590, 2665, 2740, 2840, 2880, 5045, 5105, 5170, 5220, 5300, 5345, 5410, 5470, 5525, 5585, 5645, 5705, 5760, 5820, 5875, 5945, 6015, 6125, 6180, 6235, 6300, 6350, 6410, 6460, 7791], Score: 0.5373307206149149
Time Series: 1: True Change Points: [ 772 2056 3521 4835 5283 6893], Found Change Points: [4825, 5005, 5050, 8330], Score: 0.3422012112177856
Time Series: 2: True Change Points: [1211 2043 2836 3529 4036], Found Change Points: [1240, 1255, 1320, 1330, 1405, 1415, 1490, 1505, 1565, 1580, 1640, 1660, 1720, 1745, 1795, 1825, 1880, 1905, 1965, 1990, 2025, 4050, 4095, 4120, 4170, 4190, 4250, 4280, 4335, 4360, 4430, 4460, 4525, 4555, 4625, 4655, 4715, 4750, 4805, 4840, 4905, 4935, 4964], Score: 0.4033679934797768
Time Series: 3: True Change Points: [ 739 1433 2044 2875 3589], Found Change Points: [2030, 2055, 2075, 

<h3>BOCD</h3>

In [6]:
all_cps_true=[]
all_cps_found=[]
all_ts_len=[]
# Iterate over each time series
for _,(cps_true,ts) in tqdm(df.iterrows()):
    # Fit the BinaryClaSPSegmentation algorithm and predict change points
    cps_found = bocd(ts,len(cps_true))
    all_cps_true.append(cps_true)
    all_cps_found.append(cps_found)
    all_ts_len.append(ts.shape[0])

evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,"BOCD")

0it [00:00, ?it/s]

10it [03:16, 19.61s/it]


Mean Covering: 0.4758520019207311
Mean F-measure: 1.0
Mean NAB (Standard): 45.196
Mean NAB (LowFP): 39.7
Mean NAB (LowFN): 47.297


<h3>Fluss</h3>

In [ ]:
all_cps_true=[]
all_cps_found=[]
all_ts_len=[]
for _,(cps_true,ts) in tqdm(df.iterrows()):
    window_size = 50
    n_cps = len(cps_true)
    # Using fluss function
    cps_found = fluss(ts, window_size, n_cps)
    all_cps_true.append(cps_true)
    all_cps_found.append(cps_found)
    all_ts_len.append(ts.shape[0])

evaluate_and_save(all_cps_true,all_cps_found,all_ts_len,"Fluss")

250it [23:06,  5.55s/it]
d:\Time_Series_Segmenation\benchmark\Evaluating\../../benchmark/utils\evaluation.py:378: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\../../benchmark/utils\evaluation.py:378: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\../../benchmark/utils\evaluation.py:378: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\../../benchmark/utils\evaluation.py:378: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_

Mean Covering: 0.6477299605369065
Mean F-measure: 0.9939809523809525
Mean NAB (Standard): 43.828401826484004
Mean NAB (LowFP): 38.87054794520546
Mean NAB (LowFN): 45.60296803652966


d:\Time_Series_Segmenation\benchmark\Evaluating\../../benchmark/utils\evaluation.py:378: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\../../benchmark/utils\evaluation.py:378: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)
d:\Time_Series_Segmenation\benchmark\Evaluating\../../benchmark/utils\evaluation.py:378: RuntimeWarning: invalid value encountered in scalar divide
  results[profile_name] = round(100*(matrix[0,t]-matrix[1,t])/(matrix[2,t]-matrix[1,t]), 2)


<h3>Save the results</h3>

In [ ]:
# Convert dictionary to DataFrame
results_df = pd.DataFrame(results).T.reset_index()

# Rename the 'index' column to 'name_algo'
results_df = results_df.rename(columns={'index': 'name_algo'})
# save the file
results_df.to_csv('results/results_human_activity_segmentation.csv', index=False)

print("DataFrame saved as 'results_human_activity_segmentation.csv'")
# Display the DataFrame
results_df

DataFrame saved as 'results_human_activity_segmentation.csv'


,name_algo,Covering,F-measure,NAB (Standard),NAB (LowFP),NAB (LowFN)
0,BinaryClaSPSegmentation,0.61252,0.721318,40.931127,36.440986,42.537324
1,Fluss,0.64773,0.993981,43.828402,38.870548,45.602968
